In [72]:
%pylab inline --no-import-all
from scipy.optimize import fsolve
from scipy.stats import chi2
from numpy import exp, sqrt
from scipy.ndimage.morphology import *
from scipy.ndimage import *

Populating the interactive namespace from numpy and matplotlib


In [44]:
def expectedEuler(u, n1, n2):
    return chi2.cdf(u, 1) + np.exp(-u/2)*(n1+n2*np.sqrt(u))

In [45]:
def equations(p,obs1, obs2,u1, u2):
    n1,n2 = p
    return (obs1-expectedEuler(u1,n1,n2), obs2-expectedEuler(u2,n1,n2))

In [49]:
n1, n2 = fsolve(equations, (1,1), args=(33.5, 94.6, 0., 1.))

print n1, n2

print equations((n1,n2),33.5, 94.6, 0., 1.)

33.5 121.343467521
(0.0, -1.4210854715202004e-14)


In [47]:
def testEquations(p):
    x, y = p
    return (x+y**2-4, np.exp(x) + x*y - 3)

x, y =  fsolve(testEquations, (1, 1))

print testEquations((x, y))

(4.4508396968012676e-11, -1.0512035686360832e-11)


Calculate Euler characteristic

In [139]:
#create Fig 3 of http://arxiv.org/pdf/1105.4355v1.pdf
#need padding to avoid roll-over
a = np.zeros((7,7))
a[1,2]=1
a[1,3]=1
a[2,1]=1
a[2,2]=1
a[2,3]=1
a[2,4]=1

a[3,1]=1
a[3,2]=1
a[3,3]=1
a[3,4]=1
a[3,5]=1

a[4,1]=1
a[4,2]=1
a[4,3]=1
a[4,4]=1

a[5,3]=1

a[6,0]=1
a[6,1]=1

a=a.T
#need padding to avoid roll-over in convolution
#shouldn't need to with mode=constant and cval=0. hmmm
#a=np.roll(a,1, axis=0)
#a=np.roll(a,1, axis=1)
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  1.,  1.,  0.,  1.],
       [ 0.,  1.,  1.,  1.,  1.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  0.,  1.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [140]:
face_filter=np.zeros((2,2))+1
face_filter

array([[ 1.,  1.],
       [ 1.,  1.]])

In [141]:
right_edge_filter=np.zeros((2,2))
right_edge_filter[0,0]=1
right_edge_filter[0,1]=1
right_edge_filter

array([[ 1.,  1.],
       [ 0.,  0.]])

In [142]:
bottom_edge_filter=np.zeros((2,2))
bottom_edge_filter[0,0]=1
bottom_edge_filter[1,0]=1
bottom_edge_filter

array([[ 1.,  0.],
       [ 1.,  0.]])

In [152]:
faces = convolve(a,face_filter,mode='constant')>3
n_faces = np.sum(faces)
n_faces, faces

(7, array([[False, False, False, False, False, False, False],
        [False, False,  True,  True, False, False, False],
        [False,  True,  True,  True, False, False, False],
        [False, False,  True,  True, False, False, False],
        [False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False]], dtype=bool))

In [153]:
right_edges = convolve(a,right_edge_filter,mode='constant')>1
n_edges = np.sum(right_edges)
n_edges, right_edges

(11, array([[False, False,  True,  True, False, False, False],
        [False,  True,  True,  True, False, False, False],
        [False,  True,  True,  True,  True, False, False],
        [False, False,  True,  True, False, False, False],
        [False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False]], dtype=bool))

In [154]:
bottom_edges = convolve(a,bottom_edge_filter,mode='constant')>1
n_edges+=np.sum(bottom_edges)
np.sum(bottom_edges), bottom_edges

(12, array([[False, False, False, False, False,  True, False],
        [False,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True, False, False, False],
        [False,  True,  True,  True, False, False, False],
        [False, False,  True, False, False, False, False],
        [False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False]], dtype=bool))

In [155]:
vertices = a>0
n_vertices = np.sum(vertices)

In [158]:
EulerCharacteristic = n_vertices-n_edges+n_faces
print '%d-%d+%d=%d' %(n_vertices,n_edges,n_faces,EulerCharacteristic)

18-23+7=2
